# Exploring Tennis Attributes and Tournament Outcomes

Authors: A.J. Madison, Robert Silber, and Spencer Tillman

This project is exploring the attributes of tennis players and how they placed in various tournaments throughout their career.

## Literature Review

Literature review

## Project Background

Information about purpose of the project

## Gather and Clean Data

Importing data and cleaning it

## Exploratory Data Analysis

EDA and characteristics of the data

## Feature Engineering

Look for any feature engineering opportunities to build on the existing data

## Linear Regression

Create multiple models for linear regression

## Conclusion

Findings

## Next Steps

Any follow on analysis that could be performed